In [1]:
import numpy as np, pandas as pd, string
from string import digits
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

# Load and preprocess dataset
lines = pd.read_csv("Hindi_English_Truncated_Corpus.csv", encoding='utf-8')
lines = lines[lines['source'] == 'ted'][['english_sentence', 'hindi_sentence']].dropna().drop_duplicates()
lines = lines.sample(n=25000, random_state=42)

def clean_text(text):
    exclude = set(string.punctuation)
    text = ''.join(ch for ch in text if ch not in exclude)
    text = text.translate(str.maketrans('', '', digits))
    return text.strip().lower()

lines['english_sentence'] = lines['english_sentence'].apply(clean_text)
lines['hindi_sentence'] = lines['hindi_sentence'].apply(clean_text)
lines['hindi_sentence'] = lines['hindi_sentence'].apply(lambda x: 'start_ ' + x + ' _end')

# Tokenization
eng_tokenizer = Tokenizer()
eng_tokenizer.fit_on_texts(lines['english_sentence'])
eng_seq = eng_tokenizer.texts_to_sequences(lines['english_sentence'])
eng_vocab_size = len(eng_tokenizer.word_index) + 1

hin_tokenizer = Tokenizer(filters='')
hin_tokenizer.fit_on_texts(lines['hindi_sentence'])
hin_seq = hin_tokenizer.texts_to_sequences(lines['hindi_sentence'])
hin_vocab_size = len(hin_tokenizer.word_index) + 1

# Sequence Padding
max_eng_len = 20
max_hin_len = 20

encoder_input = pad_sequences(eng_seq, maxlen=max_eng_len, padding='post')
decoder_input = pad_sequences(hin_seq, maxlen=max_hin_len, padding='post')

# Decoder target: shift decoder_input left by 1
decoder_target = np.zeros((decoder_input.shape[0], decoder_input.shape[1], 1), dtype='float32')
decoder_target[:, 0:-1, 0] = decoder_input[:, 1:]

# Model Architecture
latent_dim = 256

# Encoder
encoder_inputs = Input(shape=(None,))
enc_emb = Embedding(eng_vocab_size, latent_dim)(encoder_inputs)
enc_outputs, state_h, state_c = LSTM(latent_dim, return_state=True)(enc_emb)
encoder_states = [state_h, state_c]

# Decoder
decoder_inputs = Input(shape=(None,))
dec_emb_layer = Embedding(hin_vocab_size, latent_dim)
dec_emb = dec_emb_layer(decoder_inputs)
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(dec_emb, initial_state=encoder_states)
decoder_dense = Dense(hin_vocab_size, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Model
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer='rmsprop', loss='sparse_categorical_crossentropy')
model.fit([encoder_input, decoder_input], decoder_target,
          batch_size=64, epochs=20, validation_split=0.2)

# Inference Models
# Encoder inference
encoder_model_inf = Model(encoder_inputs, encoder_states)

# Decoder inference
decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_inf_emb = dec_emb_layer(decoder_inputs)
decoder_inf_outputs, state_h_inf, state_c_inf = decoder_lstm(decoder_inf_emb, initial_state=decoder_states_inputs)
decoder_inf_outputs = decoder_dense(decoder_inf_outputs)
decoder_model_inf = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_inf_outputs, state_h_inf, state_c_inf]
)

# Reverse mapping
reverse_eng = {v: k for k, v in eng_tokenizer.word_index.items()}
reverse_hin = {v: k for k, v in hin_tokenizer.word_index.items()}

# Translation function
def translate(sentence):
    sentence = clean_text(sentence)
    seq = eng_tokenizer.texts_to_sequences([sentence])
    padded = pad_sequences(seq, maxlen=max_eng_len, padding='post')
    states = encoder_model_inf.predict(padded)

    target_seq = np.zeros((1, 1))
    target_seq[0, 0] = hin_tokenizer.word_index['start_']

    decoded = []
    for _ in range(max_hin_len):
        output_tokens, h, c = decoder_model_inf.predict([target_seq] + states)
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_word = reverse_hin.get(sampled_token_index, '')

        if sampled_word == '_end' or sampled_word == '':
            break

        decoded.append(sampled_word)

        target_seq = np.zeros((1, 1))
        target_seq[0, 0] = sampled_token_index
        states = [h, c]

    return ' '.join(decoded)

# Test
print("English: And")
print("Hindi:", translate("And"))


Epoch 1/20
313/313 [==============================] - 184s 580ms/step - loss: 3.1643 - val_loss: 2.8641
Epoch 2/20
313/313 [==============================] - 205s 657ms/step - loss: 2.7333 - val_loss: 2.6998
Epoch 3/20
313/313 [==============================] - 213s 681ms/step - loss: 2.5834 - val_loss: 2.6224
Epoch 4/20
313/313 [==============================] - 206s 657ms/step - loss: 2.4861 - val_loss: 2.5761
Epoch 5/20
313/313 [==============================] - 209s 669ms/step - loss: 2.4064 - val_loss: 2.5285
Epoch 6/20
313/313 [==============================] - 211s 673ms/step - loss: 2.3340 - val_loss: 2.5015
Epoch 7/20
313/313 [==============================] - 199s 636ms/step - loss: 2.2662 - val_loss: 2.4789
Epoch 8/20
313/313 [==============================] - 213s 681ms/step - loss: 2.2021 - val_loss: 2.4655
Epoch 9/20
313/313 [==============================] - 210s 672ms/step - loss: 2.1421 - val_loss: 2.4582
Epoch 10/20
313/313 [==============================] - 210s 672m